### 1.data load

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

#  data load
df2019 = pd.read_csv("./data/2019_사업보고서_01_재무상태표_연결_20200623.txt", sep = "\t", encoding = "cp949")

In [77]:
df2019[df2019["회사명"] == "레드로버"]

,종목코드,회사명,시장구분,업종,업종명,결산월,결산기준일,보고서종류,항목코드,항목명,당기
22484,[060300],레드로버,코스닥시장상장법인,591,"영화, 비디오물, 방송프로그램 제작 및 배급업",12,2019-12-31,사업보고서,ifrs-full_AssetsAbstract,자산 [abstract],NaN
22485,[060300],레드로버,코스닥시장상장법인,591,"영화, 비디오물, 방송프로그램 제작 및 배급업",12,2019-12-31,사업보고서,ifrs-full_CurrentAssets,유동자산,9.473623e+09
22486,[060300],레드로버,코스닥시장상장법인,591,"영화, 비디오물, 방송프로그램 제작 및 배급업",12,2019-12-31,사업보고서,ifrs-full_CashAndCashEquivalents,현금및현금성자산,9.551862e+07
22487,[060300],레드로버,코스닥시장상장법인,591,"영화, 비디오물, 방송프로그램 제작 및 배급업",12,2019-12-31,사업보고서,ifrs-full_TradeAndOtherCurrentReceivables,매출채권 및 기타유동채권,8.643018e+09
22488,[060300],레드로버,코스닥시장상장법인,591,"영화, 비디오물, 방송프로그램 제작 및 배급업",12,2019-12-31,사업보고서,ifrs-full_OtherCurrentFinancialAssets,기타유동금융자산,NaN
22489,[060300],레드로버,코스닥시장상장법인,591,"영화, 비디오물, 방송프로그램 제작 및 배급업",12,2019-12-31,사업보고서,ifrs-full_CurrentTaxAssets,당기법인세자산,1.763072e+07
22490,[060300],레드로버,코스닥시장상장법인,591,"영화, 비디오물, 방송프로그램 제작 및 배급업",12,2019-12-31,사업보고서,ifrs-full_Inventories,재고자산,1.000394e+08
22491,[060300],레드로버,코스닥시장상장법인,591,"영화, 비디오물, 방송프로그램 제작 및 배급업",12,2019-12-31,사업보고서,dart_OtherCurrentAssets,기타유동자산,6.174157e+08
22492,[060300],레드로버,코스닥시장상장법인,591,"영화, 비디오물, 방송프로그램 제작 및 배급업",12,2019-12-31,사업보고서,ifrs-full_NoncurrentAssets,비유동자산,1.856741e+10
22493,[060300],레드로버,코스닥시장상장법인,591,"영화, 비디오물, 방송프로그램 제작 및 배급업",12,2019-12-31,사업보고서,dart_NonCurrentAvailableForSaleFinancialAssets,비유동매도가능금융자산,NaN


### 2. 불필요한 변수 제거

In [25]:
# 필요 없는 변수 제거
def delete_col(data):
    data.drop("통화", axis = 1, inplace = True)
    data.drop("Unnamed: 15", axis = 1, inplace = True)
    data.drop("전기", axis = 1, inplace = True)
    data.drop("전전기", axis = 1, inplace = True)
    data.drop("재무제표종류", axis = 1, inplace = True)
    return data

df2019 = delete_col(df2019)

### 3. 연속형변수(당기, 전기, 전전기)  dtype 변경

In [26]:
def str_to_float(data):
    import numpy as np
    
    data["당기"] = data["당기"].str.replace(",", "")
#     data["전기"] = data["전기"].str.replace(",", "")
#     data["전전기"] = data["전전기"].str.replace(",", "")
    data["당기"] = data["당기"].astype(np.float32)
    return data

df2019 = str_to_float(df2019)

### 4. 항목명 변수 처리

In [27]:
def duplication_check(data):
    # 항목코드 중복 확인
    entity_코드 = []
    entity_명 = []
    without_entity_코드 = []
    without_entity_명 = []

    # 각 리스트에 코드 및 코드명 append(소문자 처리)
    for idx in range(len(data)):
        if "entity" in data["항목코드"][idx]:
            entity_코드.append(data["항목코드"][idx].lower())
            entity_명.append(data["항목명"][idx].lower())
        else:
            without_entity_코드.append(data["항목코드"][idx].lower())
            without_entity_명.append(data["항목명"][idx].lower())

    # !entity 코드 딕셔너리
    dic_without_entity = {re.findall("_\w{1,}", without_entity_코드[idx])[0].lower():[] for idx in range(len(without_entity_코드))}

    for i, j in zip(without_entity_코드, without_entity_명):
        dic_without_entity[re.findall("_\w{1,}", i)[0].lower()].append(j)

    check_without_entity = {i:len(set(j)) for i, j in dic_without_entity.items()}
    
    # entity 코드 딕셔너리
    dic_entity = {entity_코드[idx]:[] for idx in range(len(entity_코드))}
    
    for i, j in zip(entity_코드, entity_명):
        dic_entity[i].append(j)
        
    check_entity = {i:len(set(j)) for i, j in dic_entity.items()}
    
    return entity_코드, entity_명, without_entity_코드, without_entity_명, dic_without_entity, dic_entity

entity_코드, entity_명, without_entity_코드, without_entity_명, dic_without_entity, dic_entity = duplication_check(df2019)

### 5. 공시된 항목명, 항목코드 가져오기(금융감독원 재무제표 양식)

In [28]:
def extract_element_id(data):
    element_idx_lst_entity = []
    element_idx_lst_without_entity = []
    
    for idx in range(len(data)):
        if "entity" in data["항목코드"][idx]:
            element_idx_lst_entity.append(data["항목코드"][idx])
        else:
            element_idx_lst_without_entity.append(data["항목코드"][idx])
            
    entity = sorted(set(element_idx_lst_entity), reverse = True)
    without_entity = sorted(set(element_idx_lst_without_entity), reverse = True)
    
    return entity, without_entity

a, b = extract_element_id(df2019)
pd.Series(b).to_csv("./2019항목명.csv", encoding = "euc-kr", index = False)

elementid = pd.read_excel("./data/재무제표양식.xlsx", encoding = "utf-8", sheet_name = "BS1")
# '한글 Label', 'Element ID'
df_element = elementid[['한글 Label', 'Element ID']]
df_element = df_element[df_element["Element ID"].notnull()]

df_element.columns = df_element.columns.str.replace(" ", "_")
lst_element = df_element["Element_ID"].unique()

re_lst_element = []
for i in lst_element:
    a = re.findall("_\w{1,}", i)[0].lower()
    re_lst_element.append(a)
 
dic_element = {i:[] for i in re_lst_element}
for i, j in zip(re_lst_element, df_element["한글_Label"]):
    dic_element[i].append(j)

### 5-1. 항목명 entity 값 처리

In [34]:
lst_idx = []
for i in range(len(df2019)):
    if "총계" in df2019["항목명"][i] and "entity" in df2019["항목코드"][i]:
        lst_idx.append(i)

In [ ]:
# 처리 대상 항목명 추출
set(df2019[["항목코드", "항목명"]].iloc[lst_idx,]["항목명"])
# '   자본의 총계', '자본총계'
# '부채총계'
# '자본과 부채의 총계'
# '자산총계'

In [ ]:
for i in lst_idx:
    if df2019["항목명"][i] == '부채총계':
        df2019["항목코드"][i] = "ifrs-full_Liabilities"
        
    elif df2019["항목명"][i] == '자본과 부채의 총계':
        df2019["항목코드"][i] = "ifrs-full_EquityAndLiabilities"
        
    elif df2019["항목명"][i] == '자산총계':
        df2019["항목코드"][i] = "ifrs-full_Assets"
        
    else:
        df2019["항목코드"][i] = "ifrs-full_Equity"

### 6. entity 제외 항목명 변수 전처리

In [64]:
for idx in range(len(df2019)):
    if "entity" not in df2019["항목코드"][idx]:
        df2019["항목명"][idx] = dic_element[re.findall("_\w{1,}", df2019["항목코드"][idx])[0].lower()][0]

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### 7. 정제된 데이터 프레임 만들기 

In [65]:
# DB 테이블에 적용할 D.F
# 회사이름 추출
lst_company = df2019["회사명"].unique()
element_value = []
for i in dic_element.values():
    element_value.append(i[0])
    
# columns
col = list(df2019.columns[0:8]) + element_value
check_col = list(dic_element.keys())

#  해당년도 데이터 행길이
company_dic = {i:[] for i in lst_company}
day = []

for i in lst_company:
    company_dic[i].append(df2019[df2019["회사명"] == i]["결산기준일"].unique())
    day.append(len(df2019[df2019["회사명"] == i]["결산기준일"].unique()))
    
row = sum(day)

bon = np.zeros([row,len(col)]) + np.NAN

# D.F
last_df = pd.DataFrame(bon, columns = col)

### 8. 데이터 입력

In [67]:
dic_standard = {i:list(df2019[df2019["회사명"] == i]["결산기준일"].unique()) for i in lst_company}
last_company = []
last_standard  = []
for key, value in dic_standard.items():
    for index in range(len(value)):
        last_company.append(key)
        last_standard.append(value[index])
        
cnt = 0

for idx in range(len(last_company)):

    a = df2019[(df2019["회사명"] == last_company[idx]) & (df2019["결산기준일"] == last_standard[idx])]
    a.reset_index(drop = True, inplace = True)
    for i in range(8):
        last_df.iloc[cnt, i] = a.iloc[0, i]

    for idx3 in range(len(a)):
        if "entity" not in a["항목코드"][idx3]:
            loc = dic_element[re.findall("_\w{1,}", a["항목코드"][idx3])[0].lower()][0]
            last_df.loc[cnt, loc] = a["당기"][idx3]
    cnt += 1

### 9. 2018데이터 입력

In [ ]:
# 2018년 변수 처리
df = pd.read_csv("./data/2019_사업보고서_01_재무상태표_연결_20200623.txt", sep = "\t", encoding = "cp949")

def str_to_float(data):
    import numpy as np
    
    data["당기"] = data["당기"].str.replace(",", "")
    data["전기"] = data["전기"].str.replace(",", "")
    data["전전기"] = data["전전기"].str.replace(",", "")
    data["당기"] = data["당기"].astype(np.float32)
    data["전기"] = data["전기"].astype(np.float32)
    data["전전기"] = data["전전기"].astype(np.float32)
    return data

df = str_to_float(df)

# 필요 없는 변수 제거
def delete_col(data):
    data.drop("통화", axis = 1, inplace = True)
    data.drop("Unnamed: 15", axis = 1, inplace = True)
    data.drop("당기", axis = 1, inplace = True)
    data.drop("전전기", axis = 1, inplace = True)
    data.drop("재무제표종류", axis = 1, inplace = True)
    return data

df = delete_col(df)


# 항목명 변수 전처리
lst_idx = []
for i in range(len(df)):
    if "총계" in df["항목명"][i] and "entity" in df["항목코드"][i]:
        lst_idx.append(i)


for i in lst_idx:
    if df["항목명"][i] == '부채총계':
        df["항목코드"][i] = "ifrs-full_Liabilities"
        
    elif df["항목명"][i] == '자본과 부채의 총계':
        df["항목코드"][i] = "ifrs-full_EquityAndLiabilities"
        
    elif df["항목명"][i] == '자산총계':
        df["항목코드"][i] = "ifrs-full_Assets"
        
    else:
        df["항목코드"][i] = "ifrs-full_Equity"
        
        
for idx in range(len(df)):
    if "entity" not in df["항목코드"][idx]:
        df["항목명"][idx] = dic_element[re.findall("_\w{1,}", df["항목코드"][idx])[0].lower()][0]




# DB 테이블에 적용할 D.F
# 회사이름 추출
lst_company = df["회사명"].unique()
element_value = []
for i in dic_element.values():
    element_value.append(i[0])
    
# columns
col = list(df.columns[0:8]) + element_value
check_col = list(dic_element.keys())

#  해당년도 데이터 행길이
company_dic = {i:[] for i in lst_company}
day = []

for i in lst_company:
    company_dic[i].append(df[df["회사명"] == i]["결산기준일"].unique())
    day.append(len(df[df["회사명"] == i]["결산기준일"].unique()))
    
row = sum(day)

bon = np.zeros([row,len(col)]) + np.NAN

# D.F
add_df = pd.DataFrame(bon, columns = col)


# 정제된 데이터프레임 생성
dic_standard = {i:list(df[df["회사명"] == i]["결산기준일"].unique()) for i in lst_company}
last_company = []
last_standard  = []
for key, value in dic_standard.items():
    for index in range(len(value)):
        last_company.append(key)
        last_standard.append(value[index])
        
cnt = 0

for idx in range(len(last_company)):

    a = df[(df["회사명"] == last_company[idx]) & (df["결산기준일"] == last_standard[idx])]
    a.reset_index(drop = True, inplace = True)
    for i in range(8):
        add_df.iloc[cnt, i] = a.iloc[0, i]

    for idx3 in range(len(a)):
        if "entity" not in a["항목코드"][idx3]:
            loc = dic_element[re.findall("_\w{1,}", a["항목코드"][idx3])[0].lower()][0]
            add_df.loc[cnt, loc] = a["전기"][idx3]
    cnt += 1

add_df["결산기준일"] = add_df["결산기준일"].str.replace("2019", "2018")

last_df = pd.concat([last_df, add_df]).reset_index(drop = True)

### 10. 2017 데이터 입력

In [ ]:
# 2017년 변수 처리
df = pd.read_csv("./data/2019_사업보고서_01_재무상태표_연결_20200623.txt", sep = "\t", encoding = "cp949")

def str_to_float(data):
    import numpy as np
    
    data["당기"] = data["당기"].str.replace(",", "")
    data["전기"] = data["전기"].str.replace(",", "")
    data["전전기"] = data["전전기"].str.replace(",", "")
    data["당기"] = data["당기"].astype(np.float32)
    data["전기"] = data["전기"].astype(np.float32)
    data["전전기"] = data["전전기"].astype(np.float32)
    return data

df = str_to_float(df)

# 필요 없는 변수 제거
def delete_col(data):
    data.drop("통화", axis = 1, inplace = True)
    data.drop("Unnamed: 15", axis = 1, inplace = True)
    data.drop("당기", axis = 1, inplace = True)
    data.drop("전기", axis = 1, inplace = True)
    data.drop("재무제표종류", axis = 1, inplace = True)
    return data

df = delete_col(df)

# 항목명 변수 전처리
lst_idx = []
for i in range(len(df)):
    if "총계" in df["항목명"][i] and "entity" in df["항목코드"][i]:
        lst_idx.append(i)


for i in lst_idx:
    if df["항목명"][i] == '부채총계':
        df["항목코드"][i] = "ifrs-full_Liabilities"
        
    elif df["항목명"][i] == '자본과 부채의 총계':
        df["항목코드"][i] = "ifrs-full_EquityAndLiabilities"
        
    elif df["항목명"][i] == '자산총계':
        df["항목코드"][i] = "ifrs-full_Assets"
        
    else:
        df["항목코드"][i] = "ifrs-full_Equity"
        
        
for idx in range(len(df)):
    if "entity" not in df["항목코드"][idx]:
        df["항목명"][idx] = dic_element[re.findall("_\w{1,}", df["항목코드"][idx])[0].lower()][0]

# DB 테이블에 적용할 D.F
# 회사이름 추출
lst_company = df2019["회사명"].unique()
element_value = []
for i in dic_element.values():
    element_value.append(i[0])
    
# columns
col = list(df.columns[0:8]) + element_value
check_col = list(dic_element.keys())

#  해당년도 데이터 행길이
company_dic = {i:[] for i in lst_company}
day = []

for i in lst_company:
    company_dic[i].append(df[df["회사명"] == i]["결산기준일"].unique())
    day.append(len(df[df["회사명"] == i]["결산기준일"].unique()))
    
row = sum(day)

bon = np.zeros([row,len(col)]) + np.NAN

# D.F
add_df = pd.DataFrame(bon, columns = col)


dic_standard = {i:list(df[df["회사명"] == i]["결산기준일"].unique()) for i in lst_company}
last_company = []
last_standard  = []
for key, value in dic_standard.items():
    for index in range(len(value)):
        last_company.append(key)
        last_standard.append(value[index])
        
cnt = 0

for idx in range(len(last_company)):

    a = df[(df["회사명"] == last_company[idx]) & (df["결산기준일"] == last_standard[idx])]
    a.reset_index(drop = True, inplace = True)
    for i in range(8):
        add_df.iloc[cnt, i] = a.iloc[0, i]

    for idx3 in range(len(a)):
        if "entity" not in a["항목코드"][idx3]:
            loc = dic_element[re.findall("_\w{1,}", a["항목코드"][idx3])[0].lower()][0]
            add_df.loc[cnt, loc] = a["전전기"][idx3]
    cnt += 1

add_df["결산기준일"] = add_df["결산기준일"].str.replace("2019", "2017")

last_df = pd.concat([last_df, add_df]).reset_index(drop = True)

In [76]:
last_df.to_csv("./data/2017_2019_without_entity.csv", encoding = "euc-kr", index = False)